In [95]:
import socket
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from dragonfly import maximise_function
import re

#Number of images to average 
N = 5

#initial hexapod starting location
movehexinx_initial = b"setxpos>>-4.8"
movehexiny_initial = b"setypos>>-2.5000"
movehexinz_initial = b"setzpos>>2.000"
movehexapodtooutpos = b"setypos>>20.000"

#ypos= 20 out , ypos = -2.5 in

In [39]:
#move hexapod to its initial in position
udp_client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
hexapod_address = ('192.168.15.16', 65529)
udp_client.sendto(movehexinx_initial, hexapod_address)
time.sleep(30)
udp_client.sendto(movehexiny_initial, hexapod_address)
time.sleep(30)
udp_client.sendto(movehexinz_initial, hexapod_address)

14

In [98]:
def opt_align(hexy):
    hexycommand =bytes("setypos>>"+str(hexy[0]),'ascii')
    print(hexycommand)
    udp_client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    hexapod_address = ('192.168.15.16', 65529)
    udp_client.sendto(hexycommand, hexapod_address)
    time.sleep(5)
    
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address_probecampost = ('192.168.15.24', 65280)
    client.connect(server_address_probecampost)

    subcriptionstring3 = b'Wait>>MaxCounts'
    SubcriptionCmdLength3 = len(subcriptionstring3)
    sizepack3 = struct.pack('>i', SubcriptionCmdLength3)
    client.sendall(sizepack3 + subcriptionstring3)
    

    size = struct.unpack('>i', client.recv(4))[0]  # Extract the msg size from four bytes - mind the encoding
    str_data = client.recv(size)
    
    data = re.findall(r'\d+', str_data.decode('ascii'))
    #print('Data size: %s Data value: %s' % (size, str_data.decode('ascii')))
    
    meancounts_campost = np.int(data[2])
    client.close()
    
    print(meancounts_campost)
    
    return meancounts_campost

In [99]:
domain = [[-3,-1]]
max_capital = 10
min_val, min_pt, history = maximise_function(opt_align, domain, max_capital,opt_method='bo')

maxV = np.max(history.query_true_vals)
posV = history.curr_opt_points[np.argmax(history.query_true_vals)]

r = opt_align(maxV)
print('Best:'+str(maxV+', '+str(posV)))


Optimisation with bo(ei-ucb-ttei-add_ucb) using capital 10.0 (return_value)
b'setypos>>-2.5353527846385657'


KeyboardInterrupt: 